# 实验四 深度学习实验——基于人工神经网络的机器人地点识别

# 一、实验目的

1. 理解深度学习在图像分类任务中的基本原理和应用场景，尤其是针对机器人地点识别问题。
   
2. 掌握全连接神经网络的构建与使用，将其作为图像分类器，并与传统机器学习分类器（如朴素贝叶斯、KNN、决策树）进行对比分析。
   
3. 了解典型卷积神经网络进行特征提取的过程，并结合全连接神经网络实现图像分类，体会深度学习技术在特征提取和分类方面的优势。
   
4. 通过代码实现，熟悉深度神经网络的构建、训练、测试及部署流程，积累实际编程经验。

# 二、实验数据

采用与实验三相同的室内场景数据集，包含 6 个不同场景的图片数据，数据集已分为训练集和测试集，以便进行模型训练和测试。数据集的目录结构如下：

```plaintext
dataset
├── train
dataset/
├── 类别1/
│   ├── 图片1.jpg
│   ├── 图片2.jpg
│   └── ...
├── 类别2/
│   ├── 图片1.jpg
│   ├── 图片2.jpg
│   └── ...
└── ...

每个场景类别对应一个独立的文件夹，文件夹内包含该场景下的图片样本。文件夹名称即为类别标签，每个类别文件夹下存在的图片样本支持格式为.jpg、.png、.jpeg等。

训练集和测试集的划分比例为 8:2，即训练集包含 2437 张图片样本，测试集包含 616 张图片样本。划分结果保存在两个文件中，分别为train.txt和test.txt。

根据train.txt和test.txt文件中的路径信息，可以加载图片数据并进行后续的处理和分析。具体代码如下：

In [ ]:
import os
import numpy as np
import paddle
import paddle.nn as nn
import paddle.optimizer as optimizer
from paddle.io import Dataset, DataLoader
from paddle.vision import transforms
from paddle.vision.models import resnet34
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

In [ ]:
# 设置随机种子，保证结果可复现
paddle.seed(42)
np.random.seed(42)

# 设置设备
paddle.set_device('gpu' if paddle.is_compiled_with_cuda() else 'cpu')
device = paddle.get_device()
print(f"Using device: {device}")

In [ ]:
from imageDataset import ImageDataset

# 定义数据变换
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # 缩放图像为128x128
    transforms.ToTensor(),         # 转换为张量
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 标准化
])

# 创建数据集
train_dataset = ImageDataset(
    txt_file='train.txt',
    root_dir='dataset',
    transform=transform
)

test_dataset = ImageDataset(
    txt_file='test.txt',
    root_dir='dataset',
    transform=transform
)

在上面的代码中，从train.txt和test.txt中读取了训练集和测试集的图片路径和标签后，使用ImageDataset类来进行图像样本的读入，同时对读入的图像进行缩放，缩放为统一的尺寸为128×128。

# 三、实验步骤

## 1. 全连接神经网络分类器

### 特征提取与分类器设计

加载了训练数据集和测试数据集以后，首先需要对图像进行预处理，以及必要的数据增强操作。而在当前步骤中，为了方便与前一次实验的结果进行对比，依然省略的图像预处理和数据增强的操作，只是简单的将图像按照先行后列的方式展开为特征向量。如下图所示。

![将图像展开为向量](将图像展开为向量.png)



接下来，使用全连接神经网络构造一个分类器。该分类器包含有两个隐层，一个输出层，隐层神经元的数量可以通过参数hidden_size来设定，而且第二个隐层的神经元数量为第一个隐层神经元数量的一半，输出层神经元的数量为第二个隐层神经元数量的一半。两个隐层的激活函数均为ReLU函数，并且设定隐层神经元随机失活的概率为0.5。

In [ ]:
'''
# 定义神经网络模型
class ImageClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(ImageClassifier, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)  # 添加dropout防止过拟合
        self.fc2 = nn.Linear(hidden_size, hidden_size // 2)
        self.fc3 = nn.Linear(hidden_size // 2, num_classes)
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x
'''        

In [ ]:
# TODO：请添加百度paddle框架的代码
# 定义神经网络模型
？？？？？？？？？？？？？？？？？？？？？？？？？？？？

为了更清楚的将全连接神经网络的结构展示出来，对超参数的具体数值进行制定。其中，隐层神经元的数量指定为512，那么按照前面的网络结构定义，第一层隐层有512个神经元，第二层隐层有256个神经元，输出层8个神经元。这里输出层神经元的数量与类别标签的数量一致。

In [ ]:
# 设置参数
batch_size = 32
num_epochs = 20
learning_rate = 0.001
hidden_size = 512

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# 获取类别数量
num_classes = len(train_dataset.classes)
print(f"Number of classes: {num_classes}")
print(f"Classes: {train_dataset.classes}")

# 计算输入大小 (3通道 * 128 * 128)
input_size = 3 * 128 * 128

# 创建模型
model = ImageClassifier(input_size, hidden_size, num_classes)
print(model)

在上面的代码中，构造的整体网络结构可以表示为下图：

<img src="全连接神经网络分类器结构.png" alt="全连接神经网络分类器的结构" width="500" height="300">

### 模型训练

全连接神经网络的训练采用梯度下降算法，寻找损失函数的最小值。由于处理的是图像分类任务，因此损失函数一般采用交叉熵损失(Cross Entropy Loss)。而小批量梯度下降算法为了避免陷入局部极小值，加快算法收敛速度，也有不同的改进算法，包括了动量法(Momentum)，自适应梯度法(如RMSProp，AdaGrad等)，以及Adam算法。目前，初级神经网络炼丹师都会采用Adam算法，因为它是动量法与自适应梯度法的结合，收敛速度快，效果也不错。

在模型训练过程中，由于采用的是小批量梯度下降算法，因此需要指定批量(Batch Size)的大小，以及梯度下降算法的学习率(Learning Rate)。这两个超参数，已经在之前的代码中指定过了。


In [ ]:
from trainAndEvaluateModel_pp import train_model, evaluate_model

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optim = optimizer.Adam(learning_rate=learning_rate, parameters=model.parameters())
    
 # 训练模型
print("\nTraining model...")
train_losses, train_accs = train_model(model, train_loader, criterion, optim, num_epochs)

In [ ]:
from plotFilgures_pp import plot_training_process, plot_confusion_matrix

# 绘制训练过程
plot_training_process(train_losses, train_accs)

### 模型评估

实验在测试集上进行模型的评估，并制作混淆矩阵，打印测试报告。

In [ ]:
 # 评估模型
print("\nEvaluating model...")
test_loss, test_acc, all_preds, all_labels = evaluate_model(model, test_loader, criterion)
    

In [ ]:
# 绘制混淆矩阵
plot_confusion_matrix(all_labels, all_preds, train_dataset.classes)

In [ ]:
# 打印分类报告
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=train_dataset.classes))

**实验思考**

将这里得到的实验结果与实验三中没有使用SIFT算法得到的实验结果进行对比，看看哪种分类器的性能更强，为什么，从什么指标看出来的？

(双击此处写出你的答案)

### 模型部署

如果你对这个模型的分类性能还满意，那么可以将模型保存为.path文件，然后在其它需要使用模型的地方首先定义好模型的结构，然后使用torch.load()函数将模型参数加载进来，就可以直接使用模型进行推理预测了。

In [ ]:
# 保存模型
#paddle.save(model.state_dict(), 'image_classifier_model.pdparams')
#print("Model saved as 'resnet_classifier_model.pdparams'")

## 2. 卷积神经网络

失败是成功的老娘。道路坎坷，应该远胜于一帆风顺，所以卧薪尝胆，才能有后来的百二秦关终归楚。而又或者是因为秦关归楚得过于轻易，所以霸王殒命在大江之畔。说到底，成功也好，失败也好，保持一个时刻冷静的头脑，胜不骄，败不馁，这才是最重要的。我们的故事写到这里已经将要终章，然后自上次实验以来，似乎没有什么实验结果是可以让我们满意的，百分制三四十的分类准确率，如果应用于实际的机器人地点识别，与猜闷儿，几无区别，这也是为什么一直没有使用CoppeliaSim去对机器人地点识别进行仿真的原因。那么问题到底出在哪里呢？

回顾图像分类的全过程：首先是数据集准备，实验中使用的是一个经过阉割以后的数据集，因为百度AI Studio上传不了500M以上的文件，因此就到有些类别的数据样本数量非常少，看看前面的混淆矩阵，分得最差的三个类别健身房、博物馆、办公室，恰恰是图像样本最少的三个类别，这就为分类性能的提高制造了天然的障碍。

其次是数据预处理。如果数据集中图像样本的数量少，那么就应该在这一步想办法变出更多的图像样本来，那么有什么办法可以“变”出更多的图像样本呢？答案是数据增强。数据增强是指在不改变图像内容的前提下，通过对图像进行一系列的变换，来生成新的图像样本，从而增加数据集的规模。这一些列图像增强措施包括旋转、缩放、平移、裁剪、翻转、色彩变换、噪声添加等等。然而，在之前的实验中，我们是否采用了图像增强的办法？没有！

再次是特征提取。直接将图像按照先行后列的方式展开成特征向量，实在是太没有技术含量了，而且效果也不好。对比一下实验三中两次使用朴素贝叶斯分类器的经历，不难发现，使用SIFT算法进行图像特征提取以后，再进行分类，分类性能可以提升很多。这说明什么呢？说明任何努力不会白费的，都会是我们距离成功更近一点。

最后是分类器的选择，KNN，决策树，朴素贝叶斯，全连接神经网络，这是我们已经尝试过的几种分类器，孰优孰劣，认真做过实验的同学肯定都知道。如果没有认真做实验呢？看看他们出现的年代顺序，也能够大致猜出个一二来。

好了，问题分析得差不多了，既然故事行将结束，那么我们就一起设计一个圆满的结局吧。


#### 数据增强

为了实验结果对比的公平，还是使用之前一直在使用的数据集。然而，如前所述，这个数据集有天然的问题，阻碍了分类性能的提升。既然数据集本身不能做修改，那么我们就退而求其次，从数据预处理的角度着手，通过数据增强的方式，使图像样本尽可能丰富，于是我们就要开始施展魔法了！


In [ ]:
# 设置参数
batch_size = 32
num_epochs = 20
learning_rate = 0.001

# 设置设备
#paddle.set_device('gpu' if paddle.is_compiled_with_cuda() else 'cpu')
#device = paddle.get_device()
#print(f"Using device: {device}")

# 定义训练数据变换（包含数据增强）
train_transform = transforms.Compose([
    transforms.Resize((144, 144)),  # 稍大一些的尺寸，便于随机裁剪
    transforms.RandomCrop(128),    # 随机裁剪
    transforms.RandomHorizontalFlip(),  # 随机水平翻转
    transforms.RandomRotation(10),  # 随机旋转
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # 颜色抖动
    transforms.ToTensor(),         # 转换为张量
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 标准化
])

看看上面的代码，就会发现所谓数据增强，其手段也就是随机剪裁、随机水平翻转、随机旋转、颜色抖动而已，这些措施真的能够提升分类性能吗？让我们拭目以待。

需要注意的是，数据增强的方法只是针对训练集的，而验证集和测试集则不需要进行数据增强。

In [ ]:
# 定义测试数据变换（不包含数据增强）
test_transform = transforms.Compose([
    transforms.Resize((128, 128)),  # 缩放图像为128x128
    transforms.ToTensor(),         # 转换为张量
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 标准化
])

# 创建数据集
train_dataset = ImageDataset(
    txt_file='train.txt',
    root_dir='dataset',
    transform=train_transform  # 使用带数据增强的变换
)

test_dataset = ImageDataset(
    txt_file='test.txt',
    root_dir='dataset',
    transform=test_transform  # 使用不带数据增强的变换
)

#### 特征提取与分类器设计

特征提取是对图像所蕴含信息的再挖掘，之前通过使用SIFT图像特征提取方法，确实是使朴素贝叶斯算法的分类性能得到了提升。但是，SIFT特征提取算法毕竟是一种人工设计的特征提取方法，而卷积神经网络的出现，使得的特征的自动提取成为了可能。这里，不是说人工设计的特征提取方法就一定不好，只是人的认知总是存在一定局限性的，而当我们用自己的经验去设计图像特征提取器时，这个特征提取器的工作场景也就必然会存在一定的局限性。

经典的卷积神经网络都有哪些呢？AlexNet， VGGNet， GoogLeNet， ResNet等。ResNet，还记得2015年的ImageNet竞赛吗？Top5错误率第一次低于人类的，就是这个ResNet。正是遥想公瑾当年，羽扇纶巾，谈笑间，樯橹灰飞烟灭。那么，急用这个ResNet来提取图像特征吧。

由于这个网络模型太经典了，所以根本就不用自己搭建，直接导入一个34层的ResNet就可以。除了34层，ResNet还有110层，152层，182层等，如下图所示，网络模型的层数越多，其提取的特征就越丰富，但是相应的，其训练的时间就越长。

<img src="resnet表格.png" alt="resnet的结构" width="500" height="300">

分类器，仍然采用全连接神经网络分类器，因为如果没猜错的话，这个分类器是目前我们遇到的性能最好的分类器了。其结构仍然是两个隐层和一个输出层构成，两个隐层神经元的个数依次为512和256，输出层神经元的个数与类别标签的个数一致。因此，需要对ResNet模型的输出层做出一些改动，使的ResNet的输出可以接入全连接神经网络分类器。这样我们网络的整体结构就可以画成：

<img src="整体网络结构.png" alt="整体网络结构" width="500" height="300">



In [ ]:
'''
# 定义基于ResNet的神经网络模型
class ResNetClassifier(nn.Module):
    def __init__(self, num_classes, pretrained=True):
        super(ResNetClassifier, self).__init__()
        # 加载预训练的ResNet模型作为特征提取器
        # 使用ResNet-34
        self.feature_extractor = models.resnet34(pretrained=pretrained)
        
        # 获取ResNet最后一层的输出特征数
        feature_size = self.feature_extractor.fc.in_features
        
        # 移除ResNet的全连接层
        self.feature_extractor.fc = nn.Identity()
        
        # 添加自定义的三层全连接网络
        self.classifier = nn.Sequential(
            nn.Linear(feature_size, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        # 使用ResNet提取特征
        features = self.feature_extractor(x)
        # 通过全连接层进行分类
        return self.classifier(features)
'''

In [ ]:
# TODO：将上面的代码转换为百度paddlepaddle的代码
# 定义基于ResNet的神经网络模型
？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？？

In [ ]:
# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# 获取类别数量
num_classes = len(train_dataset.classes)
print(f"Number of classes: {num_classes}")
print(f"Classes: {train_dataset.classes}")

# 创建基于ResNet的模型
model = ResNetClassifier(num_classes=num_classes, pretrained=True)
print(model)

#### 模型训练

模型训练依然采用梯度下降算法，使用交叉熵作为损失函数，作为初级炼丹师，直接使用adam实现小批量梯度下降。

In [ ]:
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optim = optimizer.Adam(learning_rate=learning_rate, parameters=model.parameters())

# 训练模型
print("\nTraining model...")
train_losses, train_accs = train_model(model, train_loader, criterion, optim, num_epochs)

# 绘制训练过程
plot_training_process(train_losses, train_accs)

#### 模型评估

在测试集上对训练好的模型进行评估，让我们再来看看混淆矩阵的主对角线是否会被彻底点亮。

In [ ]:
# 评估模型
print("\nEvaluating model...")
test_loss, test_acc, all_preds, all_labels = evaluate_model(model, test_loader, criterion)

# 绘制混淆矩阵
plot_confusion_matrix(all_labels, all_preds, train_dataset.classes)

In [ ]:
# 打印分类报告
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=train_dataset.classes))

#### 模型部署

这个模型你满意吗？如果满意，那么把模型给参数保存下来，需要使用的时候，只需要定义好模型结构，然后加载训练好的模型参数就可以预测推理了。

In [ ]:
# 保存模型
#paddle.save(model.state_dict(), 'resnet_classifier_model.pdparams')
#print("Model saved as 'resnet_classifier_model.pdparams'")

如果你对这个模型还不是很满意，那么想一想还可以再做点什么来提升它的性能？

无论满意或是不满意，时间到了，故事就该结束了。可是故事真的结束了吗？也许故事才刚刚开始，还好，它开始于一个有所追求的地方，开始于追求更高识别性能的地方，开始于2025年的春天，开始于一间教室，在这间教室里，一位教师站在讲台上说：“让我们一起来学习一下这门课，这门课的名字是‘人工智能基础’……”